In [1]:
import googleapiclient.discovery
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime
from pytz import timezone
import os

load_dotenv()

## config
KEY_API = os.getenv('chave_api_youtube')
SERVICE_NAME='youtube'
VERSION='v3'
## ids playlist and channels
ID_PLAYLIST_CAVALEIRO_DA_LUA = "PLCG86DHec6YGDUIHqtDT8IJzyCh8gJu6l"
ID_PLAYLIST_MISS_MARVEL = "PL4M4DVGB6vMLJrIH93c3K3I8lasy1DQ9o"
ID_PLAYLIST_GAVIAO = "PL4M4DVGB6vMLTTjA1fIZfIW-sS5TCsKW9"
ID_PLAYLIST_DOCTOR_STRANGE = "PL4M4DVGB6vMJ7XsOssB5hLZP4LdqaFJMx"
ID_PLAYLIST_ETERNOS = "PL4M4DVGB6vMJP8KMJv65xCDvnmiRyKAPk"
ID_PLAYLIST_LOKI = "PL4M4DVGB6vMLPi7pkOFjHq-tBDUPZRuAh"
ID_PLAYLIST_VIUVA_NEGRA = "PL4M4DVGB6vMKkYpJEYxM4zSUOJKkZuaVN"
ID_PLAYLIST_FALCAO_SOLDADO = "PL4M4DVGB6vMLekuB-F-v1Vn5I136CVSxf"
ID_PLAYLIST_SANG_CHI = "PL4M4DVGB6vMIvUWrzOzL-d4WT1xmVZdxC"
ID_PLAYLIST_WHAT_IF = "PL4M4DVGB6vMIESCt3JOuHItJjV1L1-sJA"
ID_PLAYLIST_SPIDER_MAN_3 = "PL4M4DVGB6vMKFHKUm3aBTmNNIRoARvvlN"
ID_PLAYLIST_WANDA_VISION = "PL4M4DVGB6vMIbzvWMrFgtFeg6nvnl7Aym"


CHANNEL_ID_DISNEY_PLUS = 'UCApaSzvP6jM9rfs8UbcqD1g'
CHANNEL_ID_MARVEL = "UCItRs-h8YU1wRRfP637614w"
PLAYLIST_NAME = "miss marvel"
VIDEO_ID = 'kHhJlL4d5Vo'
# next_page_token = None

## instanciando
youtube = googleapiclient.discovery.build(
    serviceName=SERVICE_NAME, version=VERSION, developerKey=KEY_API
)

def pega_dados_totais(id_channel=CHANNEL_ID_DISNEY_PLUS):
    res = youtube.channels().list(part='statistics', id=id_channel).execute()
    return res

def pega_dados_playlist(pagetoken=None, id_playlist=ID_PLAYLIST_CAVALEIRO_DA_LUA, max_results=50):
    res = youtube.playlistItems().list(part=['snippet', 'contentDetails'], playlistId=id_playlist, maxResults=max_results, pageToken=pagetoken).execute()
    return res

def pega_dados_video(id_video):
    res = youtube.videos().list(part='statistics', id=id_video, maxResults=50).execute()
    return res

def pega_json_comments(id_video=VIDEO_ID, max_results=50, pagetoken=None):
    res = youtube.commentThreads().list(part=['snippet', 'replies'], videoId=id_video,  maxResults=max_results, pageToken=pagetoken).execute() 
    return res

In [16]:
for i in pega_json_comments()['items']:
    print(i.get('snippet').get('totalReplyCount'))
    o = i.get('snippet').get('totalReplyCount')
    if int(o) > 0:
        print(i.get('snippet'))
        break

0
0
0
1
{'videoId': 'kHhJlL4d5Vo', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'Z1UX-EwJ3xjb-TegD28inV5SE9E', 'id': 'Ugy1Q2OtiCbZQLut7ud4AaABAg', 'snippet': {'videoId': 'kHhJlL4d5Vo', 'textDisplay': 'Estamos de olho.. Uma gotinha fora do balde JÁ ERA.. O publico é peão de verdade... BOOOOOOOLSONARISTAAAAAAAS viventes das realidades do norte e centro oeste brasileiro a DÉCADAS. Uma lacradinha e JÁ ERA.', 'textOriginal': 'Estamos de olho.. Uma gotinha fora do balde JÁ ERA.. O publico é peão de verdade... BOOOOOOOLSONARISTAAAAAAAS viventes das realidades do norte e centro oeste brasileiro a DÉCADAS. Uma lacradinha e JÁ ERA.', 'authorDisplayName': 'Gustavo Alfa', 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AKedOLQxgFupzmJbWaHNye5knklwKIwh5TXe_Wlz9jUiKg=s48-c-k-c0x00ffffff-no-rj', 'authorChannelUrl': 'http://www.youtube.com/channel/UC9XDvLX20_t3OFqTtugLEHQ', 'authorChannelId': {'value': 'UC9XDvLX20_t3OFqTtugLEHQ'}, 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, 

In [23]:
def video_comments(video_id ='SecT4r0-BqE'):
    dados = []
    next_page_token = None
    ## laço que me retorna para dados uma lista com json de cada comentário
    while True:
        json = pega_json_comments(id_video=video_id, pagetoken=next_page_token)
        # pega o page token para puxar a próxima página
        next_page_token = json.get('nextPageToken')
        dados_brutos = json['items']
        dados += dados_brutos
        if next_page_token is None:
            break
    ## lista que armazena os dados finais retirados do json
    lista_dados = []
    for comments in dados:
        video_id = comments['snippet']['topLevelComment']['snippet']['videoId']
        author_name = comments['snippet']['topLevelComment']['snippet']['authorDisplayName']
        author_img = comments['snippet']['topLevelComment']['snippet']['authorProfileImageUrl']
        like_counts = comments['snippet']['topLevelComment']['snippet']['likeCount']
        text = comments['snippet']['topLevelComment']['snippet']['textOriginal']
        dict_dados = {'video_id': video_id, 'author_img': author_img, 'author_name': author_name, 'like_counts': like_counts, 'text': text}
        lista_dados.append(dict_dados)
    return lista_dados

In [20]:
video_comments(VIDEO_ID)

[{'video_id': 'kHhJlL4d5Vo',
  'author_img': 'https://yt3.ggpht.com/ytc/AKedOLSOEazMx0yzUFTBKF1Twzh2rj4pBbLCZO9D7pXO2_EJTSzTHMt1Ops-ngzguAQr=s48-c-k-c0x00ffffff-no-rj',
  'author_name': 'Marilda Ribeiro da silva',
  'like_counts': 0,
  'text': 'O ator que vai interpretar Jove parece um boneco de cera.'},
 {'video_id': 'kHhJlL4d5Vo',
  'author_img': 'https://yt3.ggpht.com/qSXstq2w_j9EIxMRu37jdxoDRSS8A112U5O92AL4hOYxRKpPo0LanK9oODUddUKk3IvwbroVCw=s48-c-k-c0x00ffffff-no-rj',
  'author_name': 'Fabiane Lima',
  'like_counts': 0,
  'text': 'Capricharam nesse elenco 👏👏👏'},
 {'video_id': 'kHhJlL4d5Vo',
  'author_img': 'https://yt3.ggpht.com/ytc/AKedOLS5zkEgHxm34oH5OkVgbhq5-13YvZWgrlGBcg=s48-c-k-c0x00ffffff-no-rj',
  'author_name': 'Ricardo Guilherme Schiavoni',
  'like_counts': 0,
  'text': 'Pena que é na globo... não dá para assistir.'},
 {'video_id': 'kHhJlL4d5Vo',
  'author_img': 'https://yt3.ggpht.com/ytc/AKedOLQxgFupzmJbWaHNye5knklwKIwh5TXe_Wlz9jUiKg=s48-c-k-c0x00ffffff-no-rj',
  'author_

In [21]:
## pega os comentários de uma lista de vídeos
def pega_comments_videos(list_videos_id):
    comments_list = []
    for video_id in list_videos_id:
        comments_list += video_comments(video_id)
    df = pd.DataFrame(comments_list)
    fuso_horario = timezone('America/Sao_Paulo')
    df['loading_ts'] = datetime.datetime.now().astimezone(fuso_horario).strftime(format='%Y-%m-%d %H:%M:%S')
    df['loading_ts'] = df.loading_ts.astype('datetime64')
    return df

In [25]:
pega_comments_videos([VIDEO_ID])

,video_id,author_img,author_name,like_counts,text,loading_ts
0,kHhJlL4d5Vo,https://yt3.ggpht.com/ytc/AKedOLSOEazMx0yzUFTB...,Marilda Ribeiro da silva,0,O ator que vai interpretar Jove parece um bone...,2022-04-04 10:43:34
1,kHhJlL4d5Vo,https://yt3.ggpht.com/qSXstq2w_j9EIxMRu37jdxoD...,Fabiane Lima,0,Capricharam nesse elenco 👏👏👏,2022-04-04 10:43:34
2,kHhJlL4d5Vo,https://yt3.ggpht.com/ytc/AKedOLS5zkEgHxm34oH5...,Ricardo Guilherme Schiavoni,0,Pena que é na globo... não dá para assistir.,2022-04-04 10:43:34
3,kHhJlL4d5Vo,https://yt3.ggpht.com/ytc/AKedOLQxgFupzmJbWaHN...,Gustavo Alfa,0,Estamos de olho.. Uma gotinha fora do balde JÁ...,2022-04-04 10:43:34
4,kHhJlL4d5Vo,https://yt3.ggpht.com/ytc/AKedOLQHXM9XFgMC3Gw1...,Eglair Siqueira,0,Só tenho amor por essa novela !!,2022-04-04 10:43:34
...,...,...,...,...,...,...
271,kHhJlL4d5Vo,https://yt3.ggpht.com/YS6wDiveb4S7SUN6V1ereyo_...,LB Novelas (Laura Novelas),12,Que Elenco INCRÍVEL,2022-04-04 10:43:34
272,kHhJlL4d5Vo,https://yt3.ggpht.com/syy-1Ue3nl1bYxCKSEoVBSR1...,Pauliane Top show,3,Novela boa,2022-04-04 10:43:34
273,kHhJlL4d5Vo,https://yt3.ggpht.com/ZFgrbKdxxKF1OyEVth82lyZr...,Tafa Victoreli,54,"Vim do 31 de dezembro de 2022 , falar que Pant...",2022-04-04 10:43:34
274,kHhJlL4d5Vo,https://yt3.ggpht.com/_9IVO1mpmEe7HeS_dLp-Vvyz...,Victor Almeida,4,Essa novela vai ter vilões ? Se sim quais,2022-04-04 10:43:34
